# Creating and analysing the simulated datasets #
Current code includes: 
Number of transcripts: each transcript has its own SNPs, each with their own error rate, and its own mobility status.

Number of SNPs: the number of SNPs per transcript (currently fixed).

Number of replicates: We create 2 homograft files (Nhom1 and Nhom2), and then no_reps heterograft files. 

# Method A #

If a single SNP has a read depth >= min_read_thresh, then it is flagged as being mobile. If 1 or more SNPs are flagged as being mobile, then the transcript is flagged as being mobile.

# Handling SNPs #

Bayes Factors: sum them all (should we point out in the paper that this is a useful advantage of this method?)

Method A: snp_thresh SNPs need to have reads mapping to the other ecotype, in order for a mobile classification to be given for the transcript

Method B: snp_thres SNPs need to have reads mapping to the other ecotype, in order for a mobile classification to be given for the transcript

# Handling replicates #

Bayes Factors: Again, very simple. We sum across replicates

Method A: A transcript needs to have been given a mobile assignment in rep_thresh replicates, in order for a final mobile assignment to be given.

Method B: A transcript needs to have been given a mobile assignment in rep_thresh replicates, in order for a final mobiel assigment to be given.

In [ ]:
import baymobil as baymob

## Running and evaluating the simulations
## Each simulation is designed to investigate the accuracy of the results as a function of some parameter. The values for this parameter must be defined in the "parameters.cfg" file

parameter_func = "N"

## Create the datasets. This function creates no_reps .csv files, storing them in the "output" folder. This will delete any previous files, so make sure you have moved anything that you still need.
baymob.simulations.create_simulated_data(parameter_func)

## Now, we load in and combine all of the replicate data, according to the above rules
df = baymob.load_data()

## Plot and compare the accuracy (TP + TN) / (TP + TN + FP + FN) of the three different methods on our simulated datasets
baymob.plot_data(df,parameter_func)


In [ ]:
import baymobil as baymob
out = baymob.run_bayes_analysis([100,10,100,10,100,10])
out

In [1]:
import pandas as pd
test_path = "test_data/"

#dfhom1 = pd.read_csv(test_path + "C-C-root-FN.csv")
#dfhom2 = pd.read_csv(test_path + "P-P-root-FN.csv")
#dfhet = pd.read_csv(test_path + "Col-FN-root-1.csv")

dfhom1_path = test_path + "C-C-root-FN.csv"
dfhom2_path = test_path + "P-P-root-FN.csv"
dfhet_path = test_path + "Col-FN-root-1.csv"

#dfhom1.columns = ["SNP","N","eco1","n"]
#dfhom2.columns = ["SNP","N","eco1","n"]
#dfhet.columns = ["SNP","N","eco1","n"]

import baymobil.baymobil as baymob

#df = baymob.run_bayes_analysis([dfhom1, dfhom2, dfhet])
baymob.run_bayes_analysis([dfhom1_path, dfhom2_path, dfhet_path])


0it [00:00, ?it/s]

Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Error rate in homograft higher than heterograft. Setting BF to -2.
Warning! Err

9it [00:16,  1.87s/it]


In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv("test_data/Col-FN-root-1_results.csv")
df[df["log10BF"]==-2]

/var/folders/t5/q1tt12hd2sg29kw894zcfbtm000b_5/T/ipykernel_56556/4121431839.py:3: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("test_data/Col-FN-root-1_results.csv")


,SNP,N,eco1,n,Nhomo1,nhomo1,Nhomo2,nhomo2,nmax,meanN2,N2max,log10BF
12627,Chr1_7453861,6221.0,6221.0,0.0,5469.0,0.0,5851.0,189.0,10,nd,nd,-2.0
12628,Chr1_7453999,6800.0,6769.0,31.0,4599.0,13.0,6669.0,166.0,10,nd,nd,-2.0
12631,Chr1_7454229,4522.0,4522.0,0.0,2631.0,0.0,3536.0,213.0,10,nd,nd,-2.0
12636,Chr1_7454812,8059.0,8039.0,20.0,8881.0,14.0,8891.0,6112.0,10,nd,nd,-2.0
13998,Chr1_8389598,1841.0,1546.0,295.0,1955.0,333.0,2638.0,42.0,10,nd,nd,-2.0
13999,Chr1_8389709,1883.0,1883.0,0.0,2060.0,3.0,2806.0,735.0,10,nd,nd,-2.0
14000,Chr1_8389728,1279.0,1278.0,1.0,1297.0,1.0,1854.0,296.0,10,nd,nd,-2.0
14006,Chr1_8390445,2333.0,2330.0,3.0,1909.0,5.0,3112.0,297.0,10,nd,nd,-2.0
14721,Chr1_9084307,5123.0,4768.0,355.0,5736.0,228.0,1482.0,108.0,10,nd,nd,-2.0
17870,Chr1_11370851,3.0,3.0,0.0,2.0,0.0,58.0,0.0,10,0.010478324933632623,1,-2.0


In [1]:
import baymobil.baymobil as baymob
results = baymob.run_bayes_analysis([2,0,58,0,3,0])

In [2]:
print(results)

(0.010478324933632623, 1, -2.0)
